In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from timm import create_model

In [4]:
# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
# Load datasets
train_dataset = datasets.ImageFolder('D:/Személyes/Tanulmányok/ELTE IK/Prog Inf MSc/4.félév/Diplomamunka/repo/mri_processing/data/kaggle_brain_tumor/Training', transform=train_transform)
test_dataset = datasets.ImageFolder('D:/Személyes/Tanulmányok/ELTE IK/Prog Inf MSc/4.félév/Diplomamunka/repo/mri_processing/data/kaggle_brain_tumor/Testing', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize DINOv2
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_lc')

Using cache found in C:\Users\Csabi/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Csabi/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Csabi/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Csabi/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [11]:
for param in model.parameters():
    param.rqueres_grad=False

In [14]:
model.linear_head = torch.nn.Linear(in_features=1920, out_features=len(train_dataset.classes), bias=True)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Tensorboard writer
writer = SummaryWriter()

In [17]:



# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Wrap train_loader with tqdm for progress visualization
        train_loader_with_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)

        for inputs, labels in train_loader_with_progress:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_loader_with_progress.set_postfix({'loss': loss.item(), 'acc': (predicted == labels).sum().item() / labels.size(0)})

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total

        writer.add_scalar('Train/Loss', train_loss, epoch)
        writer.add_scalar('Train/Accuracy', train_accuracy, epoch)

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            all_predictions = []
            all_labels = []

            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            try:
                val_loss /= len(test_loader)
                val_accuracy = val_correct / val_total
            
                val_precision = precision_score(all_labels, all_predictions, average='macro'
                val_roc_auc = roc_auc_score(all_labels, all_predictions, multi_class='ovr'))

                writer.add_scalar('Validation/Loss', val_loss, epoch)
                writer.add_scalar('Validation/Accuracy', val_accuracy, epoch)
                writer.add_scalar('Validation/ROC_AUC', val_roc_auc, epoch)
                writer.add_scalar('Validation/Precision', val_precision, epoch)
            except Exception as e:
                print(e)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    writer.close()




In [ ]:
# Train the model
train_model(model, criterion, optimizer)